In [ ]:
from langchain.chat_models import ChatOpenAI


message = "내일 날씨좀 알려줘."
llm = ChatOpenAI(
  model_name="gpt-4o-mini",
)

response = llm.predict(message)

print(response)

In [ ]:
from langchain.chat_models import ChatOpenAI


chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1 # 낮을수록 정확도가 높고, 높을수록 창의성이 높아짐
)


from langchain.schema import HumanMessage, AIMessage, SystemMessage

messages = [
  SystemMessage(content="You are a geography expert. And you only reply in Italian."),
  AIMessage(content="Ciao, mi chiamo Paolo!"),
  HumanMessage(content="What is the distance between Mexico and Thailand. Also, what is your name?")
]

chat.predict_messages(messages)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

template = PromptTemplate(
  input_variables=["country_a", "country_b"],
  template="What is the distance between {country_a} and {country_b}",
)
chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1
)

prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)

In [ ]:
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1
)

template = ChatPromptTemplate.from_messages([
  ("system", "You are a geography expert. And you only reply in {language}"),
  ("ai", "Ciao, mi chiamo {name}"),
  ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
  language="Greek",
  name="Socrates",
  country_a="Mexico",
  country_b="Thailand"
)

chat.predict_messages(prompt)


In [ ]:
from langchain.schema import BaseOutputParser


class CommonOutputParser(BaseOutputParser):
  def parse(self, text):
    items = text.strip().split(",")
    return list(map(str.strip, items))
  
  
p = CommonOutputParser()
  
p.parse("Hello,how,are,you")

ChatPromptTemplate

In [ ]:


template = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated a list of {max_items}. Do not reply with anything else."),
    ("human", "{question}")
  ]
)

prompt = template.format_messages(
  max_items=10,
  question="What are the planets?"
)

result = chat.predict_messages(prompt)

p = CommonOutputParser()

p.parse(result.content)



In [ ]:
chain = template | chat | CommonOutputParser()

chain.invoke({
  "max_items": 5,
  "question": "What are the pokemons?"
})

In [18]:



chef_template = ChatPromptTemplate.from_messages([
  ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
  ("human", "i want to cook {cuisine} food.")
])

chef_chain = chef_template | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a vegetarian chef speciailized on making traditional recipies vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
  ("human", "{recipe}")
])


veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
  "cuisine": "Indian"
})

AIMessage(content="Chana Masala is a fantastic vegetarian dish that is already suitable for a vegetarian diet, as it primarily consists of chickpeas and spices. However, if you're looking for alternatives or modifications to enhance the recipe, here are a few suggestions:\n\n### Chana Masala Recipe (Vegetarian)\n\n#### Ingredients:\n- 1 can (15 oz) chickpeas (garbanzo beans), drained and rinsed\n- 1 medium onion, finely chopped\n- 2 medium tomatoes, chopped\n- 2-3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 2 green chilies, slit (adjust to taste)\n- 1 teaspoon cumin seeds\n- 1 teaspoon coriander powder\n- 1 teaspoon cumin powder\n- 1 teaspoon garam masala\n- ½ teaspoon turmeric powder\n- ½ teaspoon red chili powder (adjust to taste)\n- Salt to taste\n- 2 tablespoons vegetable oil (can substitute with olive oil or coconut oil)\n- Fresh cilantro, chopped (for garnish)\n- Lemon wedges (for serving)\n\n#### Instructions:\n\n1. **Heat Oil**: In a large pan, heat the vegetable

In [23]:

from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
  model_name="gpt-4o-mini",
  temperature=0.1,
  callbacks=[StreamingStdOutCallbackHandler()],
  streaming=True
)

poetry_prompt = ChatPromptTemplate.from_messages({
  ("system", "너는 세상에서 가장 아름답고 멋진 시를 만드는 시인이야. 너는 프로그래밍 세상에서 특정 언어에 대한 시를 쓸 수 있어."),
  ("human", "{language}")
})

poetry_chain = poetry_prompt | chat

poetry_chain.invoke({
  "language": "Java"
})

poetry_description_prompt = ChatPromptTemplate.from_messages({
  ("system", "너는 시인의 시를 보고 설명하는 해설가야."),
  ("human", "{poetry}")
})

poetry_description_chain = poetry_description_prompt | chat

final_chain = {"poetry": poetry_chain} | poetry_description_chain


**Java의 노래**

자바의 세계, 그 깊은 바다,  
객체의 춤, 메서드의 노래,  
클래스와 인터페이스, 조화의 선율,  
코드의 미로 속, 우리는 항해하네.

강한 타입, 안전한 품,  
예외 처리로 지키는 꿈,  
멀티스레드의 리듬 속에,  
동시성의 마법, 우린 함께해.

JVM 위에서 펼쳐지는,  
플랫폼의 경계를 넘는 자유,  
"Hello, World!"의 첫 인사,  
모든 시작은 이곳에서.

컬렉션의 바다, 리스트의 물결,  
맵의 길을 따라, 데이터의 향연,  
람다의 숨결, 함수형의 미소,  
코드의 아름다움, 그 속에 담겨.

자바의 철학, "Write Once, Run Anywhere",  
어디서든 빛나는 그 가능성,  
우리는 함께 만들어가는,  
이 세상의 모든 꿈과 희망.

자바여, 너의 이름을 부르며,  
코드의 시를, 영원히 노래하리,  
우리가 만든 이 아름다운 세계,  
자바의 노래, 끝없이 이어지리.

In [ ]:
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


example_prompt = ChatPromptTemplate.from_messages([
  ("human", "What do you know abount {country}"),
  ("ai", "{answer}")
])

prompt = FewShotChatMessagePromptTemplate(
  examples=examples,
  example_prompt=example_prompt,
)

final_prompt = ChatPromptTemplate.from_messages(
  [
    ("system", "You are a geography expert. you give short answers."),
    prompt,
    ("human", "What do you know about {country}")
  ]
)

chain = final_prompt | chat

chain.invoke({"country": "Thailand"})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum\n        Currency: Thai Baht\n        ')

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector


chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]


class RandomExampleSelector(BaseExampleSelector):
    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice

        return [choice(self.examples)]


example_prompt = PromptTemplate.from_template("Human: {question}\nAI:{answer}")

example_selector = RandomExampleSelector(
    examples=examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

prompt.format(country="Brazil")
      

'Human: What do you know about France?\nAI:\n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

prompts = [
    ("intro", intro),
    ("example", example),
    ("start", start),
]


full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)


chain = full_prompt | chat

chain.invoke(
    {
        "character": "Pirate",
        "example_question": "What is your location?",
        "example_answer": "Arrrrg! That is a secret!! Arg arg!!",
        "question": "What is your fav food?",
    }
)


In [3]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


chat = ChatOpenAI(
    temperature=0.1,
    # streaming=True,
    # callbacks=[
    #     StreamingStdOutCallbackHandler(),
    # ],
)

chat.predict("How do you make italian pasta")
"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it with your hands or a rolling pin until it's about 1/4 inch thick.\n\n7. If you have a pasta machine, pass the dough through the machine, starting with the widest setting and gradually decreasing the thickness until you reach the desired thickness for your pasta. If you don't have a pasta machine, you can use a rolling pin to roll out the dough until it's thin enough.\n\n8. Once the dough is rolled out, you can cut it into your desired pasta shape. For example, you can cut it into thin strips for spaghetti or fettuccine, or you can use a pasta cutter to create shapes like farfalle (bowtie) or ravioli.\n\n9. As you cut the pasta, lay it out on a clean surface or hang it on a pasta drying rack to prevent it from sticking together.\n\n10. To cook the pasta, bring a large pot of salted water to a boil. Add the pasta and cook it until it's al dente, which means it's cooked but still slightly firm to the bite. The cooking time will vary depending on the thickness and shape of the pasta, so it's best to taste it to determine when it's done.\n\n11. Once the pasta is cooked, drain it and serve it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!"
chat.predict("How do you make italian pasta")
"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it with your hands or a rolling pin until it's about 1/4 inch thick.\n\n7. If you have a pasta machine, pass the dough through the machine, starting with the widest setting and gradually decreasing the thickness until you reach the desired thickness for your pasta. If you don't have a pasta machine, you can use a rolling pin to roll out the dough until it's thin enough.\n\n8. Once the dough is rolled out, you can cut it into your desired pasta shape. For example, you can cut it into thin strips for spaghetti or fettuccine, or you can use a pasta cutter to create shapes like farfalle (bowtie) or ravioli.\n\n9. As you cut the pasta, lay it out on a clean surface or hang it on a pasta drying rack to prevent it from sticking together.\n\n10. To cook the pasta, bring a large pot of salted water to a boil. Add the pasta and cook it until it's al dente, which means it's cooked but still slightly firm to the bite. The cooking time will vary depending on the thickness and shape of the pasta, so it's best to taste it to determine when it's done.\n\n11. Once the pasta is cooked, drain it and serve it with your favorite sauce or toppings.\n\nEnjoy your homemade Italian pasta!"

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n\n2. Crack the eggs into the well and add the salt.\n\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency. If it's too wet, add a little more flour.\n\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to work with.\n\n6. After resting, divide the dough into 

In [ ]:

from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback

chat = ChatOpenAI(
    temperature=0.1,
)


with get_openai_callback() as usage:
    a = chat.predict("What is the recipe for soju")
    b = chat.predict("What is the recipe for bread")
    print(a, "\n")
    print(b, "\n")
    print(usage)


AttributeError: 'ChatOpenAI' object has no attribute 'save'